In [ ]:
# 시뮬레이션에서 데이터 받아오기 
import pygame
import sys
import requests
import json

# 초기화
pygame.init()
pygame.joystick.init()

joystick = pygame.joystick.Joystick(0)
joystick.init()

print(f"Joystick Name: {joystick.get_name()}")
print(f"Number of Axes: {joystick.get_numaxes()}")
print(f"Number of Buttons: {joystick.get_numbuttons()}")

# Node.js 서버 URL
NODE_SERVER_URL = "http://192.168.219.58:5000/simulation_data"

# 데이터 읽기 루프
running = True
axis_values = [0] * joystick.get_numaxes()
button_states = [0] * joystick.get_numbuttons()

# 이전 상태 저장
prev_axis_values = axis_values.copy()
prev_button_states = button_states.copy()

def send_data_to_server(data):
    try:
        response = requests.post(NODE_SERVER_URL, json=data)
        if response.status_code == 200:
            print("\rData sent successfully", end='', flush=True)
        else:
            print(f"\rFailed to send data. Status code: {response.status_code}", end='', flush=True)
    except requests.exceptions.RequestException as e:
        print(f"\rError sending data: {e}", end='', flush=True)

while running:
    for event in pygame.event.get():
        if event.type == pygame.QUIT:
            running = False

        # 축 데이터 읽기 (핸들 회전, 페달 입력)
        if event.type == pygame.JOYAXISMOTION:
            axis = event.axis
            value = joystick.get_axis(axis)
            axis_values[axis] = value

        # 버튼 데이터 읽기 (기어 변경)
        if event.type == pygame.JOYBUTTONDOWN:
            button = event.button
            button_states[button] = 1
        elif event.type == pygame.JOYBUTTONUP:
            button = event.button
            button_states[button] = 0

    # 상태 변경 확인
    if axis_values != prev_axis_values or button_states != prev_button_states:
        # 데이터 준비
        data = {
            "axes": axis_values,
            "buttons": button_states
        }

        # Node.js 서버로 데이터 전송
        send_data_to_server(data)

        # 이전 상태 업데이트
        prev_axis_values = axis_values.copy()
        prev_button_states = button_states.copy()

    # 결과 출력 (한 줄로 갱신)
    output = "Axes: "
    for i, value in enumerate(axis_values):
        output += f"{i}: {value:.2f}, "
    output += "Buttons: "
    for i, state in enumerate(button_states):
        if state:
            output += f"{i}, "
    print(f"\r{output}", end='', flush=True)

    # 프레임 제어
    pygame.time.Clock().tick(30)

pygame.quit()
sys.exit()